# Generate txt file

In [ ]:
from datetime import datetime, timedelta

def get_date_strings():
    today = datetime.today()
    date_strings = [(today - timedelta(days=i)).strftime("%Y%j") for i in range(15)]
    return date_strings

def write_structure_to_file(file_path):
    year = 2024
    date_strings = get_date_strings()
    
    structure = [
        "+ /NRT/Wind/Daily/"  + date + ".nc" for date in date_strings
    ]
    structure.extend([
        "+ /NRT/P/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.extend([
        "+ /NRT/Pres/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.extend([
        "+ /NRT/RelHum/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.extend([
        "+ /NRT/SpecHum/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.extend([
        "+ /NRT/Tmin/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.extend([
        "+ /NRT/Tmax/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.extend([
        "+ /NRT/Temp/Daily/"  + date + ".nc" for date in date_strings
    ])
    structure.append("- *")
    
    with open(file_path, 'w') as file:
        for line in structure:
            file.write(line + '\n')

# Specify the file path where you want to save the structure
file_path = "file_structure_with_14_days.txt"

# Call the function to write the structure to the file
write_structure_to_file(file_path)

print(f"File '{file_path}' has been created with the specified structure.")
import subprocess
import os


dest_folder = "new_data"

def run_rclone_sync(filter_file_path):
    # Define the command to run
    command = [
        "docker//rclone", "sync", "-v", "--filter-from", filter_file_path, "--drive-shared-with-me",
        "google:/MSWX_V100", dest_folder
    ]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
        print("rclone sync command executed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

# Path to the filter file generated before
filter_file_path = "file_structure_with_14_days.txt"

# Call the function to run the rclone sync command
run_rclone_sync(filter_file_path)


In [ ]:
import subprocess
import os


dest_folder = "new_data"

def run_rclone_sync(filter_file_path):
    # Define the command to run
    command = [
        "data/rclone", "sync", "-v", "--filter-from", filter_file_path, "--drive-shared-with-me",
        "google:/MSWX_V100", dest_folder
    ]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
        print("rclone sync command executed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

# Path to the filter file generated before
filter_file_path = "file_structure_with_14_days.txt"

# Call the function to run the rclone sync command
run_rclone_sync(filter_file_path)


# Get training data

In [1]:
import csv
from datetime import datetime, timedelta

def get_unique_dates_from_csv(csv_file_path):
    """Extract unique observation dates from the CSV file"""
    unique_dates = set()
    
    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if 'observed_on' in row and row['observed_on']:
                # Parse the date part (ignore timezone)
                date_str = row['observed_on'].split(" ")[0]
                try:
                    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
                    unique_dates.add(date_obj)
                except ValueError:
                    print(f"Could not parse date: {row['observed_on']}")
                    continue
    
    return sorted(list(unique_dates))


def generate_file_structure_from_csv(csv_file_path, output_file_path):
    """
    Generate file structure for all dates in CSV with 15 days before each observation,
    for both MSWX_V100/PAST and MSWX_V100/NRT.
    """
    unique_dates = get_unique_dates_from_csv(csv_file_path)
    print(f"Found {len(unique_dates)} unique observation dates")
    
    all_files = set()
    variables = ['Wind', 'P', 'Pres', 'RelHum', 'SpecHum', 'Tmin', 'Tmax', 'Temp']
    
    for obs_date in unique_dates:
        # For each observation date, get 15 days before (including the observation day)
        for i in range(15):
            target_date = obs_date - timedelta(days=i)
            date_string = target_date.strftime("%Y%j")
            
            for variable in variables:
                # Add entries for both NRT and PAST
                all_files.add(f"+ /NRT/{variable}/Daily/{date_string}.nc")
                all_files.add(f"+ /PAST/{variable}/Daily/{date_string}.nc")
    
    # Convert to sorted list
    file_list = sorted(list(all_files))
    file_list.append("- *")  # Exclude everything else
    
    with open(output_file_path, 'w') as file:
        for line in file_list:
            file.write(line + '\n')
    
    print(f"Generated {len(file_list)-1} unique file entries")
    print(f"File structure saved to: {output_file_path}")
    
    return file_list


# Test
csv_file_path = "data/negative_samples_within_land_10k_with_coords_topography.csv"
output_file_path = "climate_files_from_csv_neg.txt"
file_structure = generate_file_structure_from_csv(csv_file_path, output_file_path)


Could not parse date: 12/30/2020 0:00
Could not parse date: 7/24/2009 0:00
Could not parse date: 5/16/2019 0:00
Could not parse date: 6/15/2003 0:00
Could not parse date: 12/22/2023 0:00
Could not parse date: 11/21/2006 0:00
Could not parse date: 10/19/2007 0:00
Could not parse date: 8/18/2001 0:00
Could not parse date: 6/12/2001 0:00
Could not parse date: 12/19/2017 0:00
Could not parse date: 12/16/2004 0:00
Could not parse date: 10/24/2004 0:00
Could not parse date: 7/14/2004 0:00
Could not parse date: 11/18/2013 0:00
Could not parse date: 6/5/2015 0:00
Could not parse date: 7/4/2009 0:00
Could not parse date: 4/27/2007 0:00
Could not parse date: 1/15/2009 0:00
Could not parse date: 10/12/2001 0:00
Could not parse date: 4/17/2011 0:00
Could not parse date: 3/4/2001 0:00
Could not parse date: 11/25/2004 0:00
Could not parse date: 10/16/2021 0:00
Could not parse date: 1/3/2000 0:00
Could not parse date: 5/30/2017 0:00
Could not parse date: 4/15/2014 0:00
Could not parse date: 9/25/2006

# Full working

In [ ]:
import os
import subprocess
import csv
from datetime import datetime, timedelta
from tqdm import tqdm


def get_unique_dates_from_csv(csv_file_path, limit_rows=None):
    """Extract unique observation dates from the CSV file, with optional row limit."""
    unique_dates = set()

    with open(csv_file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        total_rows = sum(1 for _ in open(csv_file_path)) - 1  # minus header
        csvfile.seek(0)

        for i, row in enumerate(tqdm(reader, total=limit_rows or total_rows, desc="Reading CSV")):
            if limit_rows is not None and i >= limit_rows:
                break

            if 'observed_on' in row and row['observed_on']:
                date_str = row['observed_on'].split(" ")[0]
                date_obj = None
                
                # Try different date formats
                date_formats = [
                    "%Y-%m-%d",      # 2022-03-02
                    "%m/%d/%Y",      # 2/3/2022
                    "%d/%m/%Y",      # 3/2/2022
                    "%Y/%m/%d",      # 2022/03/02
                    "%m-%d-%Y",      # 2-3-2022
                    "%d-%m-%Y",      # 3-2-2022
                ]
                
                for fmt in date_formats:
                    try:
                        date_obj = datetime.strptime(date_str, fmt)
                        break
                    except ValueError:
                        continue
                
                if date_obj:
                    unique_dates.add(date_obj)
                else:
                    print(f"⚠️ Could not parse date: {row['observed_on']}")
                    continue
    return sorted(list(unique_dates))


def generate_file_structure_from_csv(csv_file_path, output_file_path, limit_rows=None):
    """
    Generate file structure for all dates in CSV with 15 days before each observation,
    for both MSWX_V100/PAST and MSWX_V100/NRT.
    """
    unique_dates = get_unique_dates_from_csv(csv_file_path, limit_rows=limit_rows)
    print(f"📅 Found {len(unique_dates)} unique observation dates")

    all_files = set()
    variables = ['Wind', 'P', 'Pres', 'RelHum', 'SpecHum', 'Tmin', 'Tmax', 'Temp']

    for obs_date in tqdm(unique_dates, desc="Building file list"):
        for i in range(15):
            target_date = obs_date - timedelta(days=i)
            date_string = target_date.strftime("%Y%j")

            for variable in variables:
                all_files.add(f"+ /Past/{variable}/Daily/{date_string}.nc")
                all_files.add(f"+ /NRT/{variable}/Daily/{date_string}.nc")

    file_list = sorted(list(all_files))
    file_list.append("- *")  # exclude everything else

    with open(output_file_path, 'w') as file:
        for line in file_list:
            file.write(line + '\n')

    print(f"📝 Generated {len(file_list)-1} unique file entries")
    print(f"✅ File structure saved to: {output_file_path}")
    return file_list


def run_rclone_sync_fixed(filter_file_path, dest_folder="climate_data"):
    """Run rclone sync for both PAST and NRT using one filter file"""
    rclone_path = os.path.join("..", "docker", "rclone.exe")

    if not os.path.exists(rclone_path):
        print(f"❌ Error: rclone not found at {rclone_path}")
        return

    command = [
        rclone_path, "sync", "-v",
        "--filter-from", filter_file_path,
        "--drive-shared-with-me",
        "google:/MSWX_V100",  # root folder
        dest_folder
    ]

    print(f"▶️ Running command: {' '.join(command)}")
    try:
        subprocess.run(command, check=True)
        print("✅ rclone sync command executed successfully.")
        print(f"📂 Climate data downloaded to: {dest_folder}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error running rclone: {e}")
    except FileNotFoundError:
        print("❌ Error: rclone executable not found.")


def download_climate_data_from_csv_fixed(csv_file_path, dest_folder="climate_data", limit_rows=None):
    """Complete workflow with filter file + rclone sync"""
    filter_file_path = "climate_files_from_csv.txt"

    print("Step 1: Generating file structure from CSV...")
    generate_file_structure_from_csv(csv_file_path, filter_file_path, limit_rows=limit_rows)

    print("Step 2: Downloading climate data files...")
    run_rclone_sync_fixed(filter_file_path, dest_folder)

    return filter_file_path


if __name__ == "__main__":
    csv_file_path = "..//data/negative_samples_within_land_10k_with_coords_topography.csv"
    download_climate_data_from_csv_fixed(csv_file_path)


Step 1: Generating file structure from CSV...


Reading CSV: 100%|██████████| 5403/5403 [00:00<00:00, 168054.58it/s]


📅 Found 2744 unique observation dates


Building file list: 100%|██████████| 2744/2744 [00:00<00:00, 18711.76it/s]


📝 Generated 140032 unique file entries
✅ File structure saved to: climate_files_from_csv.txt
Step 2: Downloading climate data files...
▶️ Running command: ..\docker\rclone.exe sync -v --filter-from climate_files_from_csv.txt --drive-shared-with-me google:/MSWX_V100 climate_data
